In [1]:
from models.lightweightOB.lightwegiht_models.yolo import Model
import torch
import torch.nn as nn
def measure_model(model: nn.Module, input_tensor: torch.Tensor):
    # 파라미터 계산
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # FLOPs 계산 (ptflops 설치 필요)
    try:
        from ptflops import get_model_complexity_info
    except ImportError:
        raise RuntimeError("ptflops 설치 필요: pip install ptflops")
    
    input_shape = tuple(input_tensor.shape[1:])  # 배치 차원 제외
    macs, _ = get_model_complexity_info(model, input_shape, as_strings=False)
    flops = macs * 2  # MACs → FLOPs 변환 (1 MAC = 2 FLOPs)
    
    # 결과 출력
    print(f"Total Parameters: {total_params:,}")
    print(f"Trainable Parameters: {trainable_params:,}") 
    print(f"FLOPs: {flops:,} ({flops/1e9:.2f} GFLOPs)")



In [2]:


# cfg: 모델 구조가 정의된 YAML 파일 경로 (예: 'models/yolov5s.yaml')
# ch: 입력 채널 수 (일반적으로 3)
# nc: 클래스 개수 (예: 80)
# anchors: (선택) anchor 정보, 보통 hyp에서 가져옴

model = Model(cfg='models\lightweightOB\lightwegiht_models\yolov5n_6_0_shufv2_ca.yaml')
measure_model(model, torch.randn(1, 3, 640, 640))

c:\ProgramData\anaconda3\envs\ComputerVision\lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Model(
  395.56 k, 100.000% Params, 343.29 MMac, 98.530% MACs, 
  (model): Sequential(
    395.56 k, 100.000% Params, 343.29 MMac, 98.530% MACs, 
    (0): CBRM(
      232, 0.059% Params, 25.4 MMac, 7.289% MACs, 
      (conv): Sequential(
        232, 0.059% Params, 24.58 MMac, 7.054% MACs, 
        (0): Conv2d(216, 0.055% Params, 22.12 MMac, 6.348% MACs, 3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, 0.004% Params, 1.64 MMac, 0.470% MACs, 8, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (2): ReLU(0, 0.000% Params, 819.2 KMac, 0.235% MACs, inplace=True)
      )
      (maxpool): MaxPool2d(0, 0.000% Params, 819.2 KMac, 0.235% MACs, kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (1): Shuffle_Block(
      872, 0.220% Params, 9.27 MMac, 2.660% MACs, 
      (branch1): Sequential(
        248, 0.063% Params, 1.69 MMac, 0.485% MACs, 
        (0): Conv2d(72, 0.018% Params, 460.8 KMac, 0.132%

In [3]:
output = model(torch.randn(1, 3, 640, 640))
print(len(output))
print(output[0].shape, output[1][0].shape, output[1][1].shape, output[1][2].shape)

2
torch.Size([1, 25200, 6]) torch.Size([1, 3, 80, 80, 6]) torch.Size([1, 3, 40, 40, 6]) torch.Size([1, 3, 20, 20, 6])


In [4]:
import os
import cv2
import yaml
import torch
import random
import numpy as np
from PIL import Image
from datetime import datetime
from models import YOLOv8n


In [5]:
model.train()

Model(
  (model): Sequential(
    (0): CBRM(
      (conv): Sequential(
        (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(8, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (1): Shuffle_Block(
      (branch1): Sequential(
        (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=8, bias=False)
        (1): BatchNorm2d(8, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.03